Nuskaityti kategoriją **Meilės romanai** ir puslapių skaičių

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse

url = "https://www.knygos.lt/lt/knygos/zanras/meiles-romanai/"
r = requests.get(url)
soup = BeautifulSoup(r.text)

psl_sk = int(soup.find(class_="numbers d-flex").find_all("li")[-2].text)
kategorija = urlparse(url).path.strip("/").split("/")[-1]
print(kategorija, psl_sk)

meiles-romanai 90


Saugoti meilės romanų knygų nuorodas į tekstinį failą

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

with open("meiles_romanai_links.txt", "a+") as file:
    for page in range(1, psl_sk + 1):
        url = f"https://www.knygos.lt/lt/knygos/zanras/{kategorija}/?psl={page}"
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")

            book_elements = soup.find_all("a", class_="product-link")
            for book in book_elements:
                href = book.get("href").strip()
                file.write(href + "\n")
        else:
            print(f"Page {page} not accessible.")
        time.sleep(1)

print("Links saved to meiles_romanai_links.txt")


Links saved to meiles_romanai_links.txt


In [ ]:
import os

folder_name = "Meiles_romanai_html"

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Aplankas '{folder_name}' buvo sukurtas.")
else:
    print(f"Aplankas '{folder_name}' jau egzistuoja.")

Aplankas 'Meiles_romanai_html' buvo sukurtas.


In [ ]:
# Knygų puslapiai saugoti 2024-11-07
from urllib.parse import urlparse
import requests
import time

# Nuskaityti knygų nuorodas iš failo
with open("meiles_romanai_links.txt", "r") as file:
    books_links = [line.strip() for line in file.readlines() if line.strip()]

    for link in books_links:
        try:
            # Užkrauti knygos puslapį
            book_response = requests.get(link)
            path_parts = urlparse(link).path.strip("/").split("/")
            with open(f"{folder_name}/{path_parts[-1]}.html", "wb") as file:  #
                file.write(book_response.content)
            time.sleep(1)
        except Exception as e:
            print(f"Klaida renkant informaciją apie knygą {link}: {e}")
            time.sleep(5)  # Pristabdyti tarp užklausų
            continue

Klaida renkant informaciją apie knygą https://www.knygos.lt/lt/knygos/paskutinis-jos-pazadas/: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Klaida renkant informaciją apie knygą https://www.knygos.lt/lt/knygos/vestuviu-fejos--indre--suzydeje-jausmai--slapyvardziu-prisidengusios-zinomos-lietuviu-rasytojos-jausmu-romanu-trilogijos-trecioji-dalis--su-autografu--1ffmf/: [Errno 2] No such file or directory: 'Meiles_romanai_html/vestuviu-fejos--indre--suzydeje-jausmai--slapyvardziu-prisidengusios-zinomos-lietuviu-rasytojos-jausmu-romanu-trilogijos-trecioji-dalis--su-autografu--1ffmf.html'
Klaida renkant informaciją apie knygą https://www.knygos.lt/lt/knygos/vasara-vasari-intelektualiai-ir-jautriai-papasakota-meiles-istorija-neisvengusi-tragiskos-lemties-i-didziuosius-kino-ekranus-perkelta-paties-christopherio-menaulo/: [Errno 2] No such file or directory: 'Meiles_romanai_html/vasara-vasari-intelektual

Nuskaityti knygų nuorodas ir išsaugoti knygų viršelius aplanke

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from time import sleep

input_folder = "Meiles_romanai_html"
output_folder = "Meiles_romanu_virseliai1"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Eiti per kiekvieną failą aplanke
for file_name in tqdm(os.listdir(input_folder)):
    file_path = os.path.join(input_folder, file_name)

    # Atidaryti ir skaityti HTML failą
    with open(file_path, "r", encoding="utf-8") as file:
        html_content = file.read()

    # Sukurti BeautifulSoup objektą, kad būtų galima analizuoti HTML
    soup = BeautifulSoup(html_content, "html.parser")

    # Surasti viršelio nuorodą
    cover_img_tag = soup.find("img", itemprop="image")
    if cover_img_tag:
        cover_url = cover_img_tag["src"]
        file_extension = os.path.splitext(cover_url)[1]  # Ištraukti failo plėtinį iš viršelio URL

        # Atsisiųskite viršelio paveikslėlį
        response = requests.get(cover_url)
        if response.status_code == 200:
            # Išsaugoti viršelį su knygos pavadinimu ir tinkamu failo formatu
            safe_title = file_name.split(".")[0]
            image_path = os.path.join(output_folder, f"{safe_title}{file_extension}")

            with open(image_path, "wb") as img_file:
                img_file.write(response.content)
        else:
            print(f"Nepavyko atsisiųsti viršelio: {safe_title}")
    else:
        print(f"Viršelio nuoroda nerasta: {safe_title}")
    sleep(2)

100%|██████████| 2676/2676 [2:17:51<00:00,  3.09s/it]  
